<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>ModelOps demo - PMML with ModelOps BYOM no code</b>
</header>

![image](images/byom_meth.png) 

<h1 style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></h1>
<p style = 'font-size:16px;font-family:Arial'>
This Notebook will show you how to work with the Bring Your Own Model (BYOM) pattern and BYOM In-Vantage Scoring. This deployment pattern allows you to use whatever data science platform you want to perform model training and import back to Vantage to do a direct deployment, scoring. For understanding more about BYOM you can review official user documentation.<br>
When you use BYOM mechanisms you can deploy models directly from IDE environment, or make a proper operationalization through ModelOps. With ModelOps, you get <b>full governance</b> around your models deployment and you can <b>leverage ModelOps automations</b> for Validation, Scoring and Monitoring with an intuitive user interface that let you audit at any time all the information from your models and provide dashboards to monitor and review alerts from your deployments.<br>This notebook will cover the Operationalization of the PIMA diabetes use case with PMML standard BYOM format. <b>PMML</b> stands for Predictive Model Markup Language. PMML provides a way for analytic applications to describe and exchange predictive models produced by data mining and machine learning algorithms.

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>
    Steps in this Notebook</b></h2>
<ol style = 'font-size:16px;font-family:Arial'>
<li>Configure the Environment </li>
 <li>Connect to Vantage</li>
 <li>Train a model and export to PMML</li>
 <li>Import the PMML into ModelOps</li>
 <li>Go through Lifecycle - Evaluation (Automated Model Report)</li>
 <li>Go through Lifecycle - Approve </li>
 <li>Go through Lifecycle - Deploy (Publish and Schedule)</li>
 <li>Go through Lifecycle - Monitor (Data Drift and Performance)</li>
 <li>Configure Monitoring alert treshold (Optional) </li>
 <li>On demand Scoring from SQL (Optional)</li></ol>

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Configure the Environment</b></h1>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>


<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1.1 Libraries installation</b></h2>

<p style = 'font-size:16px;font-family:Arial'><b>A restart of the Kernel is needed to confirm changes</b>. We use -q parameter for a non-verbose log of the installation command, you may remove this parameter if you want to know all the steps of the pip installation.</p>

In [ ]:
%pip install -q teradataml==17.20.0.3 aoa==7.0.1 nyoka==4.3.0

<p style = 'font-size:16px;font-family:Arial'><b>Hint:</b><i>The easy way to restart the kernel to bring the above installed software into memory is to type zero zero (<b> 0 0 </b>). </i></p>

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1.2 Libraries import</b></h2>

In [ ]:
from teradataml import (
    create_context, 
    remove_context,
    get_context,
    get_connection,
    DataFrame,
    retrieve_byom,
    PMMLPredict,
    configure)
from teradatasqlalchemy.types import *
import os
import pandas as pd
import getpass
import logging
import sys
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from nyoka import xgboost_to_pmml

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Connect to Vantage</b></h1>

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=03_ModelOps_BYOM_PIMA_PMML;' UPDATE FOR SESSION; ''')

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Getting Data for This Demo</b></h2>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
#%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ModelOps_cloud');"        # Takes 10 seconds
%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ModelOps_local');"        # Takes 30 seconds

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Creating predictions and model table</b></p>
<p style = 'font-size:16px;font-family:Arial'>We will create a predictions table where we get our model predictions and the model table where we will upload the model created.</p>

In [ ]:
#ddl for Aoa_Byom_Models 
query = '''CREATE SET TABLE DEMO_USER.Aoa_Byom_Models 
     (
      model_version VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_type VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      project_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      deployed_at TIMESTAMP(6) DEFAULT CURRENT_TIMESTAMP(6),
      model BLOB(2097088000))
UNIQUE PRIMARY INDEX ( model_version );
'''
 
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DEMO_USER.Aoa_Byom_Models;')
    eng.execute(query)

In [ ]:
#ddl for Pima_Patient_Predictions
query = '''CREATE MULTISET TABLE DEMO_USER.Pima_Patient_Predictions 
     (
      job_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      PatientId BIGINT,
      HasDiabetes BIGINT,
      json_report CLOB(1048544000) CHARACTER SET LATIN)
PRIMARY INDEX ( job_id );;
'''
 
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DEMO_USER.Pima_Patient_Predictions;')
    eng.execute(query)

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Train a model and export to PMML</b></h1>

<p style = 'font-size:16px;font-family:Arial'>In the previous notebook <strong><a href="./02_ModelOps_PIMA_Experimentation.ipynb">Pima Experimentation</a></strong> we have done the exploratory data analysis. Now, we wil create prediction model on the Pima dataset. We are using python modules to create our model.</p>


In [ ]:
#generate a dataframe for training
train_pdf = DataFrame.from_query(f"""
SELECT 
    F.*, D.hasdiabetes 
FROM Demo_Modelops.pima_patient_features F
JOIN Demo_Modelops.pima_patient_diagnoses D
    ON F.patientid = D.patientid 
    WHERE F.patientid MOD 5 <> 0
""").to_pandas(all_rows=True)

#identify list of features and target
features = ["NumTimesPrg", "Age", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc"]
target = "HasDiabetes"

# split data into X and y
X_train = train_pdf[features]
y_train = train_pdf[target]

# create a scikitlearn pipeline for scaling and running xgb training
model = Pipeline([('scaler', MinMaxScaler()),
                  ('xgb', XGBClassifier(eta=0.2, max_depth=6))])

#run the training
model.fit(X_train, y_train)


<h2 style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.1 Convert the model to PMML</b></h2>
<p style = 'font-size:16px;font-family:Arial'>You can use the sklearn2pmml or the nyoka python libraries to convert to pmml. The nyoka is a python only package and so it is preferrable. 

In [ ]:
xgboost_to_pmml(pipeline=model, col_names=features, target_name=target, pmml_f_name="artifacts/model.pmml")

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Import into ModelOps to Operationalize</b></h1>

<p style = 'font-size:16px;font-family:Arial'>Go to the ModelOps UI and import this as a new model inside the <b>Demo</b> project.<br>Navigate to the projects screen and select the <b>Demo</b> project. Click on it to navigate to the models screen of the project.</p>

<img src="images/04_01.png" alt="Projects screen" />

<p style = 'font-size:16px;font-family:Arial'>Once there, click on the <b>DEFINE BYOM MODEL</b> button on the top right of the screen.

<img src="images/04_02.png" alt="Models screen" />

<p style = 'font-size:16px;font-family:Arial'>Create a new model with PMML format. Then Save and Import<br>Download the model.pmml file from the folder artifacts and upload into ModelOps.<br>Ensure that <b>Enable Model Monitoring</b> is checked and classification as model type is selected. Then set the table the one you have been using (probably your username) and use as Prediction Expression

```sql
CAST(CAST(json_report AS JSON).JSONExtractValue('$.predicted_HasDiabetes') AS INT)
```

<p style = 'font-size:16px;font-family:Arial'>This prediction expression can be generated with the generator: select the variable: predicted_HasDiabetes from the screen and confirm<br>

<img src="images/04_generator.png" alt="Models screen" width="500" height="500" />
<img src="images/04_generator2.png" alt="Models screen" width="500" height="500"/>
<img src="images/04_generator3.png" alt="Models screen" width="500" height="500"/>


<img src="images/04_03.png" alt="BYOM model import sidesheeet" />

<p style = 'font-size:16px;font-family:Arial'>Then click on <b>SAVE</b>, and the import job will start.

<img src="images/04_04.png" alt="BYOM model import job log" />





<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Go through Lifecycle - Evaluation (Automated Model Report)</b></h1>

<p style = 'font-size:16px;font-family:Arial'>Open the imported ModelID Lifecycle Screen. Review imported artifacts and evaluate the model with default logic (No code needed)</p>
<img src="images/04_05.png" alt="BYOM model lifecycle"/>
<p style = 'font-size:16px;font-family:Arial'>Select Evaluation dataset and run the evaluation process.<br>
                              
<img src="images/04_06.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now you can review the Model evaluation report generated with default metrics and confusion matrix plot - this can be customized with a custom evaluation logic (later in this notebook)<br>Click on View Report from ModelID Lifecycle screen.<br>

<img src="images/04_07.png" alt="Model Evaluation Report"/>

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Go through Lifecycle - Approve </b></h1>

<p style = 'font-size:16px;font-family:Arial'>Open the imported ModelID Lifecycle Screen. Click on Approve button to move forward on the next stage.<br>Include description of approval and accept</p>

<img src="images/04_08.png" alt="Approval" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. Go through Lifecycle - Deploy (Publish and Schedule)</b></h1>

<p style = 'font-size:16px;font-family:Arial'>Open the imported ModelID Lifecycle Screen. Click on Deploy button to publish Model in Vantage.</p>
<img src="images/04_09.png" alt="Model Lifecycle screen"/>

<p style = 'font-size:16px;font-family:Arial'>Now select In-Vantage engine and click next<br>

<img src="images/04_10.png" alt="Deployment Engine" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now select which Connection (here you could use a Service Connection to run in Production optionally), Database and table you want to publish your BYOM model. Use your user "demo_user" and the table "aoa_byom_models" <br>

<img src="images/04_11.png" alt="Deployment Publish" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now select if you want to schedule your model scoring, datset template to gather where you want to store your predictions and run the deploy process<br>

<img src="images/04_12.png" alt="Deployment Schedule" width="500" height="500"/>


<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>8. Go through Lifecycle - Monitor (Data Drift and Performance)</b></h1>

<p style = 'font-size:16px;font-family:Arial'>Open the Deployment and review details of the deployed model.</p>
<img src="images/04_14.png" alt="Deployment details"/>
<p style = 'font-size:16px;font-family:Arial'>You can run the Prediction job from here , click the button to run the prediction job<br>
<img src="images/04_15.png" alt="Deployment Scoring Job" />
<p style = 'font-size:16px;font-family:Arial'>You can also review the predictions, you can take this query and run into your SQL IDE or in Notebook later.<br>
<img src="images/04_16.png" alt="Deployment Predictions" width="500" height="500"/> <img src="images/04_17.png" alt="Deployment Predictions query" width="500" height="500"/>
<p style = 'font-size:16px;font-family:Arial'>Review your Feature Drift, Prediction Drift. Here ModelOps shows the comparison of the distribution of data between training and evaluation/scoring, every scoring this gets updated and different KPIs are calculated. The one we used by default for monitoring is the Population Stability Index (PSI).<br>

<img src="images/04_18.png" alt="Feature Drift" />
<img src="images/04_19.png" alt="Prediction Drift" />


<p style = 'font-size:16px;font-family:Arial'>For Performance Monitoring, we track over time the metrics of the model. For generating new metrics we need to create a new evaluation dataset and run an evaluation job.<br>

<img src="images/04_20.png" alt="Performance Drift" />

<p style = 'font-size:16px;font-family:Arial'>This has been generated using this query for evaluation dataset target:

```sql
SELECT * FROM Demo_ModelOps.pima_patient_diagnoses F WHERE F.patientid MOD 8 <> 0 
```    
<p style = 'font-size:16px;font-family:Arial'>Now the performance has changed:<br>

<img src="images/04_21.png" alt="Performance Drift change" />

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>9. Configure Monitoring alert treshold (Optional) </b></h1>

<p style = 'font-size:16px;font-family:Arial'>
We can update the Alerts configuration for the models.<br>First enable Alert from Model catalog<br>

<img src="images/04_23.png" alt="Alert Configuration" /> 

<p style = 'font-size:16px;font-family:Arial'>Now, you can change your default alerting mechanism.<br>Go to your Model, and find the Tab "Alert Configuration"<br>

<img src="images/04_22.png" alt="Alert Configuration" /> 

<p style = 'font-size:16px;font-family:Arial'>Let's change the PSI value to 0.01 instead 0.2 <br>Use Edit option:<br>

<img src="images/04_24.png" alt="Alert Configuration PSI" width="500" height="500"/> <img src="images/04_25.png" alt="Alert Configuration PSI" width="500" height="500"/> 

<p style = 'font-size:16px;font-family:Arial'>After few minutes, this has generated new alerts on the model deployed
Go to Menu -> Alert and review the alerts generated for your model<br>

<img src="images/04_26.png" alt="Alerts" />

<p style = 'font-size:16px;font-family:Arial'>Now go to ModelID and click on View Model Drift<br>

<img src="images/04_27.png" alt="Model Lifecycle View Model Drift" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now review the Feature drift, you can check this screen also from your deployments<br>
<img src="images/04_28.png" alt="Feature Drift" />

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>10. On demand Scoring from SQL (Optional)</b></h1>


<h2><b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>10.1 View Published Models</b></h2>
<p style = 'font-size:16px;font-family:Arial'>
Once deployed via ModelOps, we can view the models published to vantage by querying the table they are published to. Note this information is available via the AOA APIs also.

In [ ]:
DataFrame.from_query("SELECT TOP 10 * FROM Demo_User.aoa_byom_models WHERE model_type='PMML'").head(10)

<h2><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>10.2 On-Demand Scoring</b></h2>
<p style = 'font-size:16px;font-family:Arial'>
Configuring VAL and BYOM locations:

In [ ]:
# configure byom/val installation
configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"


<p style = 'font-size:16px;font-family:Arial'>Replace model_version with the one we have operationalized:

In [ ]:
model_version="b724c439-9c0f-4bea-89a9-e31b70958a19"

model = DataFrame.from_query(f"""
SELECT * FROM Demo_User.aoa_byom_models 
    WHERE model_version='{model_version}'
""")


preds = PMMLPredict(
        modeldata=model,
        newdata=DataFrame.from_query("SELECT * FROM Demo_User.pima_patient_features WHERE patientid MOD 5 = 0"),
        accumulate=['PatientId'])

preds.result.head(10)

In [ ]:
#SQL version
query = f"""
SELECT * FROM MLDB.PMMLPredict (
    ON (SELECT * FROM Demo_ModelOps.pima_patient_features WHERE patientid MOD 5 = 0) AS InputTable
    ON (SELECT * FROM Demo_User.aoa_byom_models 
            WHERE model_version='{model_version}') AS ModelTable DIMENSION
    USING
      Accumulate ('patientid')
) AS td;
"""

DataFrame.from_query(query).head(10)

<h1><b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>11. Cleanup</b></h1>

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'>If you are done with ModelOps usecase, please uncomment and run the below cleanup section.</p>
</div>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
# db_drop_table(table_name = 'aoa_byom_models', schema_name = 'demo_user')
# db_drop_table(table_name = 'pima_patient_predictions', schema_name = 'demo_user')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
# %run -i ../UseCases/run_procedure.py "call remove_data('DEMO_ModelOps');"        # Takes 10 seconds

In [ ]:
remove_context()

[<< Back to PIMA Experimentation](./02_ModelOps_PIMA_Experimentation.ipynb) | [Continue to PIMA ONNX BYOM >>](./04_ModelOps_BYOM_PIMA_ONNX.ipynb)

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>